Logs   
- [2023/03/08]   
  Restart this notebook if you change the scratch library


To do 
- Review the simple concept of SQL from relational in Discrete Mathematics.   
  When you create the slides of this topic, review about relation in discrete mathematics

In [1]:
from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterator
from collections import defaultdict

We try to mimic the behaviour of the standard database   
such as PostgreSQL, MySQL, SQL Server by creating from scratch    
NotQuiteABase.

This also show the power of simulation using Python. It is not fast, but
it is enough to show the principle.

## Create Table Insert

- Table is a collection of rows
- Table has an association to a fixed *schema*
- Schema consists of column names and column types 

In [2]:
  # user_id  name   num_friends
users = [[0, "Hero", 0],
         [1, "Dunn", 2],
         [2, "Sue", 3],
         [3, "Chi", 3]]

The above table can be written as SQL like the following
```sql
CREATE TABLE users (
  user_id INT NOT NULL,
  _name VARCHAR(200),
  num_friends INT)
```

and inserting one by one each row

```sql
INSERT INTO users (user_id, _name, num_friend) VALUES (0, 'Hero', 0);
```

In [3]:
# A few type aliases we'll use later
Row = Dict[str, Any]                        # A database row
WhereClause = Callable[[Row], bool]         # Predicate for a single row
HavingClause = Callable[[List[Row]], bool]  # Predicate over multiple rows

## Implementation of SQL in Python using `Table` class

In [15]:
class Table(object):  
  def __init__(self, columns: List[str], types: List[type]) -> None:
    assert len(columns) == len(types), "# of columns must == # of types"

    self.columns = columns      # Names of columns
    self.types = types          # Data types of columns
    self.rows: List[Row] = []   # (no data yet)
  
  def col2type(self, col: str) -> type:
    """Get the type of a column"""
    idx = self.columns.index(col)     # Find the index of the column
    return self.types[idx]            # and return its type.

  def insert(self, values: list) -> None:
    # Check for right # of values
    if len(values) != len(self.types):
      raise ValueError(f"You need to provide {len(self.types)} values")

    # Check for right types of values
    for value, type_ in zip(values, self.types):
      if not isinstance(value, type_) and value is not None:
        raise TypeError(f"Expected type {type_} but got value {value}")

    # Add the corresponding dict as a "row"  
    self.rows.append(dict(zip(self.columns, values)))

  def update(self, updates: Dict[str, Any], 
              predicate: WhereClause = lambda row: True):
    """First make sure the updates have valid names and types"""
    for column, new_value in updates.items():
      if column not in self.columns:
        raise ValueError(f"invalid column: {column}")

      type_ = self.col2type(column)
      if not isinstance(new_value, type_) and new_value is not None:
        raise TypeError(f"expected type {type_}, but got {new_value}")

    # Now update
    for row in self.rows:
      if predicate(row):
        for column, new_value in updates.items():
          row[column] = new_value

  def delete(self, predicate: WhereClause = lambda row: True) -> None:
    """Delete all rows matching predicate""" 
    self.rows = [row for row in self.rows if not predicate(row)]


  def select(self, keep_columns: List[str] = None, 
             additional_columns: Dict[str, Callable] = None) -> "Table":
    """
    keep_column: it specifies the names of the columns you want to keep in
      the result. If you don't supply it, the result contains all columns
    additional_column: it is a dictionary whose keys are new column names
      and whose values are function specifying how to compute the values
      of the new columns. We'll peek at the type annotations of those
      functions to figure out the types of the new columns, so the functions
      will need to have annotated return types.
    """
    if keep_columns is None:        # If no columns specified
      keep_columns = self.columns   # return all columns

    if additional_columns is None: 
      additional_columns = {}

    # New column names and types
    new_columns = keep_columns + list(additional_columns.keys())
    keep_types = [self.col2type(col) for col in keep_columns]

    # This is how to get the return type from a type annotation
    # It will crash if `calculation` doesn't have a return type.
    add_types = [calculation.__annotations__['return']
                  for calculation in additional_columns.values()]

    # Create a new table for results
    new_table = Table(new_columns, keep_types + add_types)

    for row in self.rows:
      new_row = [row[column] for column in keep_columns]
      for column_name, calculation in additional_columns.items():
        new_row.append(calculation(row))
      new_table.insert(new_row)

    return new_table

  def where(self, predicate, WhereClause = lambda row: True) -> 'Table':
    """Return only the rows that satisfy the supplied predicate""" 
    where_table = Table(self.columns, self.types)
    for row in self.rows:
      if predicate(row):
        values = [row[column] for column in self.columns]
        where_table.insert(values)
    return where_table

  def limit(self, num_rows: int) -> "Table":
    """Return only the first `num_rows` rows"""
    limit_table = Table(self.columns, self.types)
    for i, row in enumerate(self.rows):
      if i >= num_rows:
        break 
      values = [row[column] for column in self.columns]
      limit_table.insert(values)
    return limit_table

  def group_by(self, group_by_columns: List[str], aggregates: Dict[str, Callable],
               having: HavingClause = lambda group: True) -> 'Table':
    grouped_rows = defaultdict(list)

    # Populate groups
    for row in self.rows:
      key = tuple(row[column] for column in group_by_columns)
      grouped_rows[key].append(row)

    # Result table consists of group_by columns and aggregates
    new_columns = group_by_columns + list(aggregates.keys())
    group_by_types = [self.col2type(col) for col in group_by_columns]
    aggregates_types = [agg.__annotations__['return']
                        for agg in aggregates.values()]
    result_table = Table(new_columns, group_by_types + aggregates_types)

    for key, rows in grouped_rows.items():
      if having(rows):
        new_row = list(key)
        for aggregate_name, aggregate_fn in aggregates.items():
          new_row.append(aggregate_fn(rows))
        result_table.insert(new_row)

    return result_table
  
  def order_by(self, order: Callable[[Row], Any]) -> 'Table':
    new_table = self.select()     # make a copy
    new_table.rows.sort(key=order)
    return new_table

  def join(self, other_table: 'Table', left_join: bool = False) -> 'Table':
    join_on_columns = [c for c in self.columns            # columns in
                        if c in other_table.columns]      # both table

    additional_columns = [c for c in other_table.columns  # columns only
                            if c not in join_on_columns]  # in right table
    
    # all columns from left table + additional_columns from right table
    new_columns = self.columns + additional_columns
    new_types = self.types + [other_table.col2type(col)
                                for col in additional_columns]
    
    join_table = Table(new_columns, new_types)

    for row in self.rows:
      other_rows = other_table.where(
        Table.is_join(row, join_on_columns)).rows

      # Each other row that matches this one produces a result row. 
      for other_row in other_rows:
        join_table.insert([row[c] for c in self.columns] +
                          [other_row[c] for c in additional_columns])

      # If no rows match and it's a left join, output with Nones.
      if left_join and not other_rows:
        join_table.insert([row[c] for c in self.columns] +
                          [None for c in additional_columns])

    return join_table


  """The following methods treat a table like a List[Row]"""
  def __getitem__(self, idx: int) -> Row:
    return self.rows[idx]

  def __iter__(self) -> Iterator[Row]:
    return iter(self.rows)

  def __len__(self) -> int:
    return len(self.rows)

  
  def __repr__(self):
    """Pretty representation of the table: columns then rows""" 
    rows = "\n".join(str(row) for row in self.rows)
    return f"{self.columns}\n{rows}"

  @staticmethod
  def is_join(row, join_on_columns):
    return lambda other_row: all(other_row[c] == row[c] for c in join_on_columns)

Create our `Users` table

In [16]:
# Constructor requires column names and types
users = Table(['user_id', 'name', 'num_friends'], [int, str, int])
users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

print(users)

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


Testing for the above list-like API for NoQuiteABase `Table`

In [17]:
assert len(users) == 11
assert users[1]['name'] == "Dunn"

## Update

We are going to add `update` functionality to `Users` class like
the `UPDATE` command in SQL
```sql
UPDATE users
  SET num_friends = 3
  WHERE user_id = 1;
```

See the implementation inside the `Users` class above.

Testing for `update` method

In [18]:
assert users[1]['num_friends'] == 2              # Original value

users.update({'num_friends': 3},                 # Set num_friends = 3
              lambda row: row['user_id'] == 1)   # in rows where user_id == 1

assert users[1]['num_friends'] == 3              # Update value

## DELETE

- A **dangerous** way deletes every row from a table
  ```sql
  DELETE FROM users;
  ```

- **less* dangerous ways add a `WHERE` clause and deletes only rows
  that match a certain condition
  ```sql
  DELETE FROM users WHERE user_id = 1;
  ```

See the implementation inside the `Users` class above.

Testing the `delete` method
```python
# We're not actually going to run these
users.delete(lambda row: row["user_id"] == 1)     # Deletes rows with user_id == 1
users.delete()
```

## SELECT

We implement the following `SELECT`, `WHERE`, and `LIMIT` commands
in our `Table` class 

```sql
SELECT * FROM users;                              -- get the entire contents
SELECT * FROM users LIMIT 2;                      -- get the first two rows
SELECT user_id FROM users;                        -- only get specific columns
SELECT user_id FROM users WHERE name = "Dunn";    -- only get specific row
```

```sql
-- get name column and compute its length then store it into a new column
-- with name_length
SELECT LENGTH(name) AS name_length FROM users; 
```

See the implementation inside the `Users` class above.

Testing the implementation of `.select`, `.where` and `.limit` method

In [19]:
# SELECT * FROM users;
all_users = users.select()
assert len(all_users) == 11

# SELECT * FROM users LIMIT 2;
two_users = users.limit(2)
assert len(two_users) == 2

# SELECT user_id FROM USERS;
just_ids = users.select(keep_columns=["user_id"])
assert just_ids.columns == ['user_id']

# SELECT user_id FROM users WHERE name == "Dunn";
dunn_ids = (
  users
  .where(lambda row: row["name"] == "Dunn")
  .select(keep_columns=["user_id"])
)

# SELECT LENGTH(name) AS name_length FROM users;
def name_length(row) -> int: 
  return len(row["name"])

name_lengths = users.select(keep_columns=[],
                            additional_columns={"name_length": name_length})
assert name_lengths[0]['name_length'] == len("Hero")

## GROUP BY

Next, we implement `GROUP BY` to our `Table` class

Some examples using `GROUP BY`

> Find the number of users and the smallest `user_id`
> for each possible name length
>```sql
>SELECT LENGTH(name) as name_length,
>  MIN(user_id) AS min_user_id,
>  COUNT(*) AS num_users
>  FROM users
>  GROUP BY LENGTH(name)
>```

> The average number of friends for users whose names start    
> with specific letters but see only the results for letters    
> whose corresponding average is greater than 1.
>```sql
>SELECT SUBSTR(name, 1, 1) as first_letter,
>  AVG(num_friends) as avg_num_friends
>  FROM users
>  GROUP BY SUBSTR(name, 1, 1)
>  HAVING AVG(num_friends) > 1;
>```

> An example compute overall aggregates
>```sql
>SELECT SUM(user_id) as user_id_sum
>  FROM users
>  WHERE user_id > 1;
>```


Metric for `name_length`

In [20]:
def name_length(row) -> int: 
  return len(row["name"])

def min_user_id(rows) -> int:
  return min(row["user_id"] for row in rows)

def length(rows) -> int:
  return len(rows)

stats_by_length = (
  users
  .select(additional_columns={"name_length": name_length})
  .group_by(group_by_columns=["name_length"],
            aggregates={"min_user_id": min_user_id,
                        "num_users": length}) 
)
stats_by_length


['name_length', 'min_user_id', 'num_users']
{'name_length': 4, 'min_user_id': 0, 'num_users': 4}
{'name_length': 3, 'min_user_id': 2, 'num_users': 3}
{'name_length': 5, 'min_user_id': 5, 'num_users': 4}

Metric for `first_letter`

In [21]:
def first_letter_of_name(row: Row) -> str:
  return row["name"][0] if row["name"] else ""

def average_num_friends(rows: List[Row]) -> float:
  return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows: List[Row]) -> bool:
  return average_num_friends(rows) > 1

avg_friends_by_letter = (
  users
  .select(additional_columns={"first_letter": first_letter_of_name})
  .group_by(group_by_columns=["first_letter"],
            aggregates={"avg_num_friends": average_num_friends},
            having=enough_friends)
)
avg_friends_by_letter

['first_letter', 'avg_num_friends']
{'first_letter': 'H', 'avg_num_friends': 1.5}
{'first_letter': 'D', 'avg_num_friends': 2.5}
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'C', 'avg_num_friends': 2.5}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'K', 'avg_num_friends': 2.5}

Metric for `user_id_sum`

In [22]:
def sum_user_ids(rows: List[Row]) -> int:
  return sum(row["user_id"] for row in rows)

user_id_sum = (
  users
  .where(lambda row: row["user_id"] > 1)
  .group_by(group_by_columns=[],
            aggregates={"user_id_sum": sum_user_ids})
)
user_id_sum

['user_id_sum']
{'user_id_sum': 54}

## ORDER BY


We are going to implement `ORDER BY` command.
Here is an example

```sql
SELECT * FROM users
  ORDER BY name
  LIMIT 2;
```

See the implementation inside the `Users` class above.

Testing the method `.order_by` in `Users` class

In [23]:
friendliest_letters = (
  avg_friends_by_letter
  .order_by(lambda row: -row["avg_num_friends"])
  .limit(4) 
)

friendliest_letters

['first_letter', 'avg_num_friends']
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'D', 'avg_num_friends': 2.5}
{'first_letter': 'C', 'avg_num_friends': 2.5}

## JOIN

- Because SQL table cannot contain list in each row, we have to create     
  a second table that store a relation from one-to-many.        
  For example, when we work with our users' interests in Python,  we can just     
  give user a `list` containing his interests. We cannot do like that in     
  SQL table, so the typical solution is to create a second table.    
  This kind of strategy is called *redundancy*

  ```sql
  CREATE TABLE user_unterests (
    user_id INT NOT NULL,
    interest VARCHAR(10) NOT NULL
  );
  ```

  Implementation in Python

In [24]:
user_interests = Table(['user_id', 'interest'], [int, str])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

user_interests

['user_id', 'interest']
{'user_id': 0, 'interest': 'SQL'}
{'user_id': 0, 'interest': 'NoSQL'}
{'user_id': 2, 'interest': 'SQL'}
{'user_id': 2, 'interest': 'MySQL'}

`JOIN` is useful when we want to analyze (querying) items from two different     
tables, but connected by column name. In our example `users` and     
`user_interests` tables are connected by the column `user_id`

Using `JOIN`, we can join the corresponding table by their common columns

Example:
```sql
SELECT users.name
  FROM users
  JOIN user_interests
  ON users.user_id = user_interests.user_id
  WHERE user_interests.interest = "SQL"
```

```sql
SELECT users.id, COUNT(user_interests.interest) AS num_interests
  FROM users
  LEFT JOIN user_interests
  ON users.user_id = user_interests.user_id
```

The implementation of `JOIN` example

In [28]:
sql_users = (
  users
  .join(user_interests)
  .where(lambda row: row["interest"] == "SQL")
  .select(keep_columns=["name"])
)

sql_users

['name']
{'name': 'Hero'}
{'name': 'Sue'}

In [30]:
def count_interests(rows: List[Row]) -> int:
  """counts how many rows have non-None interests""" 
  return len([row for row in rows if row["interest"] is not None])

user_interest_counts = (
  users
  .join(user_interests, left_join=True)
  .group_by(group_by_columns=["user_id"],
            aggregates={"num_interests": count_interests})
)

user_interest_counts

['user_id', 'num_interests']
{'user_id': 0, 'num_interests': 2}
{'user_id': 1, 'num_interests': 0}
{'user_id': 2, 'num_interests': 2}
{'user_id': 3, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 6, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}
{'user_id': 8, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 10, 'num_interests': 0}

## Subqueries

Subquery is doing a query from a query result. Given a table and doing
query will result a subtable. From that subtable you can do query.
That is subquery.

An example of subquery
> Find the smallest `user_id` of anyone interested in SQL
>```sql
>SELECT MIN(user_id) AS min_user_id FROM
>  (SELECT user_id FROM user_interests WHERE interest = 'SQL') sql_interests;
>```

The implementation of the above example using `Table` class

In [32]:
def min_user_id(rows) -> int:
  return min(row["user_id"] for row in rows)

likes_sql_user_ids = (
  user_interests
  .where(lambda row: row["interest"] == "SQL")
  .select(keep_columns=['user_id'])
)

likes_sql_user_ids.group_by(group_by_columns=[],
                            aggregates={"min_user_id": min_user_id})

likes_sql_user_ids

['user_id']
{'user_id': 0}
{'user_id': 2}

## Indexing

- Our implementation to find a specific value and joining two table
  are not very efficient for a table which has a lot of rows.
- When we have duplicate items in `user_id`, our implementation of `Table` class
  will alow that. In practice, we won't allow that.
- Indexing of our row will solve this problem. Unfortunately this is 
  out of scope in our discussion

## Query Optimization

Recall the query to find all users who are interested in SQL:
```sql
SELECT users.name
  FROM users
  JOIN user_interests
  ON users.user_id = user_interests.user_id
  WHERE user_interests.interest = 'SQL';
```

Given two queries procedures from our `Table` class:

```python
(
  user_interests
  .where(lambda row: row["interest"] == 'SQL')
  .join(users)
  .select(["name"])
)
```

```python
(
  user_interests
  .join(users)
  .where(lambda row: row["interests"] == 'SQL')
  .select(["name"])
)
```

The first option is faster because filter-before-join is almost      
certainly more efficient, since in that case `join` has many fewer rows     
to operate on.

In SQL, we generally would'nt worry about this. We declare the commands    
to get the result we want, and leave it up to the query engine to execute   
them (and use indexes efficiently)

## NoSQL

- A recent trend in databases is toward nonrelational "NoSQL" databases.